# 2021 NMA Project Team

Anjali Srinivasan, Aaditya Prasad, Zac Wheeler

In [ ]:
# imports
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm

# for visualization
!pip install nilearn --quiet
from nilearn import plotting, datasets

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
/root/venv/lib/python3.7/site-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


## Data Retrieval

In [ ]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "./hcp"
if not os.path.isdir(HCP_DIR):
  os.mkdir(HCP_DIR)

# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 339

# The data have already been aggregated into ROIs from the Glasesr parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in sec

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated multiple times in each subject
N_RUNS_REST = 4
N_RUNS_TASK = 2

# Time series data are organized by experiment, with each experiment
# having an LR and RL (phase-encode direction) acquistion
BOLD_NAMES = [
  "rfMRI_REST1_LR", "rfMRI_REST1_RL",
  "rfMRI_REST2_LR", "rfMRI_REST2_RL",
  "tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR",
  "tfMRI_WM_RL", "tfMRI_WM_LR",
  "tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR",
  "tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR",
  "tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR",
  "tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR",
  "tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"
]

# You may want to limit the subjects used during code development.
# This will use all subjects:
subjects = range(N_SUBJECTS)

### Data Download

In [ ]:
fname = "hcp_task.tgz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/s4h8j/download/
  !tar -xzf $fname -C $HCP_DIR --strip-components=1

In [ ]:
fname = "hcp_covariates.tgz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/x5p4g/download/
  !tar -xzf $fname -C $HCP_DIR --strip-components=1

In [ ]:
fname = f"{HCP_DIR}/atlas.npz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/j5kuc/download

### Region Info

In [ ]:
regions = np.load(f"{HCP_DIR}/regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    myelin=regions[2].astype(np.float),
)

#print(len(region_info["name"]))
#print(len(region_info["network"]))
#print(len(region_info["myelin"]))

#print(region_info["name"])
#print(region_info["network"])
#print()

#names = region_info["name"]
#networks = region_info["network"]

#language_areas = []

#for i in range(len(networks)):
#  if networks[i] == "Auditory":
#    print(i, names[i])

#posterior-mu = posterior multimodal

In [ ]:
with np.load(f"{HCP_DIR}/atlas.npz") as dobj:
  atlas = dict(**dobj)

## Helper Functions (For Data Loading)

get_image_ids(name)

In [ ]:
def get_image_ids(name):
  """Get the 1-based image indices for runs in a given experiment.

    Args:
      name (str) : Name of experiment ("rest" or name of task) to load
    Returns:
      run_ids (list of int) : Numeric ID for experiment image files

  """
  run_ids = [
    i for i, code in enumerate(BOLD_NAMES, 1) if name.upper() in code
  ]
  if not run_ids:
    raise ValueError(f"Found no data for '{name}''")
  return run_ids

load_timeseries(subject, name, runs=None, concat=True, remove_mean=True)

In [ ]:
def load_timeseries(subject, name, runs=None, concat=True, remove_mean=True):
  """Load timeseries data for a single subject.
  
  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of experiment ("rest" or name of task) to load
    run (None or int or list of ints): 0-based run(s) of the task to load,
      or None to load all runs.
    concat (bool) : If True, concatenate multiple runs in time
    remove_mean (bool) : If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_tp array): Array of BOLD data values

  """
  # Get the list relative 0-based index of runs to use
  if runs is None:
    runs = range(N_RUNS_REST) if name == "rest" else range(N_RUNS_TASK)
  elif isinstance(runs, int):
    runs = [runs]

  # Get the first (1-based) run id for this experiment 
  offset = get_image_ids(name)[0]

  # Load each run's data
  bold_data = [
      load_single_timeseries(subject, offset + run, remove_mean) for run in runs
  ]

  # Optionally concatenate in time
  if concat:
    bold_data = np.concatenate(bold_data, axis=-1)

  return bold_data


load_single_timeseries(subject, bold_run, remove_mean=True)

In [ ]:
def load_single_timeseries(subject, bold_run, remove_mean=True):
  """Load timeseries data for a single subject and single run.
  
  Args:
    subject (int): 0-based subject ID to load
    bold_run (int): 1-based run index, across all tasks
    remove_mean (bool): If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_path = f"{HCP_DIR}/subjects/{subject}/timeseries"
  bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts

load_evs(subject, name, condition)

In [ ]:
def load_evs(subject, name, condition):
  """Load EV (explanatory variable) data for one task condition.

  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of task
    condition (str) : Name of condition

  Returns
    evs (list of dicts): A dictionary with the onset, duration, and amplitude
      of the condition for each run.

  """
  evs = []
  for id in get_image_ids(name):
    task_key = BOLD_NAMES[id - 1]
    ev_file = f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{condition}.txt"
    ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
    ev = dict(zip(["onset", "duration", "amplitude"], ev_array))
    evs.append(ev)
  return evs

condition_frames(run_evs, skip=0)

In [ ]:
def condition_frames(run_evs, skip=0):
  """Identify timepoints corresponding to a given condition in each run.

  Args:
    run_evs (list of dicts) : Onset and duration of the event, per run
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    frames_list (list of 1D arrays): Flat arrays of frame indices, per run

  """
  frames_list = []
  for ev in run_evs:

    # Determine when trial starts, rounded down
    start = np.floor(ev["onset"] / TR).astype(int)

    # Use trial duration to determine how many frames to include for trial
    # TR = 0.72  # Time resolution, in sec
    duration = np.ceil(ev["duration"] / TR).astype(int)

    # Take the range of frames that correspond to this specific trial
    frames = [s + np.arange(skip, d) for s, d in zip(start, duration)]

    frames_list.append(np.concatenate(frames))

  return frames_list

selective_average(timeseries_data, ev, skip=0)

In [ ]:
def selective_average(timeseries_data, ev, skip=0):
  """Take the temporal mean across frames for a given condition.

  Args:
    timeseries_data (array or list of arrays): n_parcel x n_tp arrays
    ev (dict or list of dicts): Condition timing information
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    avg_data (1D array): Data averaged across selected image frames based
    on condition timing

  """
  # Ensure that we have lists of the same length
  if not isinstance(timeseries_data, list):
    timeseries_data = [timeseries_data]
  if not isinstance(ev, list):
    ev = [ev]
  if len(timeseries_data) != len(ev):
    raise ValueError("Length of `timeseries_data` and `ev` must match.")

  # Identify the indices of relevant frames
  frames = condition_frames(ev, skip)
  print("frames shape: ", end=": ")
  print(np.array(frames).shape)
  #print(frames)

  # Select the frames from each image
  selected_data = np.zeros(9)
  for run_data, run_frames in zip(timeseries_data, frames):
    #print("run_frames before: ", end=": ")
    #print(run_frames)

    run_frames = run_frames[run_frames < run_data.shape[1]]
    #separated_data = np.array([])
    differences = np.diff(run_frames)

    print("differences:", end=" ")
    print(differences)


    #separate out trials!!! 

    print("run_frames after: ", end=": ")
    print(run_frames)
    
    selected_data.append(run_data[:, run_frames])

  # Take the average in each parcel

  # right now: taking all data in timeseries for condition, and all trial data
  # for that condition, and averaging EVERYTHING :)
  # avg_data = np.concatenate(selected_data, axis=-1).mean(axis=-1)

  #print(np.array(selected_data, dtype=object).shape)
  #print(a)
  return selected_data

## Task Analysis

### timeseries_task

In [ ]:
timeseries_task = []

# timeseries_task format: subject, then LR/RL, then parcel, then time
# 339, 2, 360, 316
for subject in subjects:
  timeseries_task.append(load_timeseries(subject, "language", concat=False))

In [ ]:
#print(np.array(timeseries_task).shape)
#print(timeseries_task[0])
#print()
#print(timeseries_task[0][0][0][0])

### Task Descriptions

MOTOR:&nbsp;cue,&nbsp;lf,&nbsp;lh,&nbsp;rf,&nbsp;rh,&nbsp;t

WM:&nbsp;&nbsp;&nbsp;&nbsp;0bk_body,&nbsp;0bk_faces,&nbsp;0bk_nir,&nbsp;0bk_placed,&nbsp;0bk_tools,&nbsp;&nbsp;2bk_body,&nbsp;2bk_faces,&nbsp;2bk_nir,&nbsp;2bk_placed,&nbsp;2bk_tools,&nbsp;0bk_cor,&nbsp;0bk_err,&nbsp;&nbsp;2bk_cor,&nbsp;2bk_err,&nbsp;&nbsp;all_bk_cor,&nbsp;all_bk_err

EMOTION:&nbsp;feat,&nbsp;neutral

GAMBLING:&nbsp;loss,&nbsp;loss_event,&nbsp;win,&nbsp;win_event,&nbsp;neut_event

LANGUAGE:&nbsp;cue, math,&nbsp;story, present_math,&nbsp;present_story,&nbsp;question_math,&nbsp;question_story,&nbsp;response_math,&nbsp;response_story

RELATIONAL:&nbsp;error,&nbsp;match,&nbsp;relation

SOCIAL:&nbsp;mental_resp,&nbsp;mental,&nbsp;other_resp,&nbsp;rnd

### Task Data Retrieval

In [ ]:
task = "language"
conditions = ["question_math", "response_math"] # "question_story", "response_story"
shift = 6

avg_qs = np.empty(shape=0)  #question story
avg_qm = np.empty(shape=0)  #question math
avg_rs = np.empty(shape=0)  #response story
avg_rm = np.empty(shape=0)  #response math
avgs = []

for subject in subjects:
  print(subject)
  # Get the average signal in each region for each condition

  # format: condition, then LR/RL (direction), then dictionary w/ trials
  # 4 trials for story, 9 trials for math
  evs = [load_evs(subject, task, cond) for cond in conditions]
  
  # conditions, 
  avgs = [selective_average(timeseries_task[subject], ev) for ev in evs]
  ## BOLD data timeseries: /V\M\----/\/\/
  ## EV data:              -------~~|****|
  ##                              ^ onset
  ##                               **** = duration
  ## skip = 2 = ~~

  
  # add subject average to avg arrays
  #avg_qs = np.append(avg_qs, avgs[0])
  #avg_qm = np.append(avg_qm, avgs[1])
  #avg_rs = np.append(avg_rs, avgs[2])
  #avg_rm = np.append(avg_rm, avgs[3])

#print(len(avgs[2][1]))
#print(evs[0][0]['onset'])
#print(np.array(avgs).shape)


0
frames shape: : (2,)
differences: [  1   1   1   1   1  13   1   1   1   1   1  10   1   1   1   1   1 114
   1   1   1   1   1  10   1   1   1   1  11   1   1   1   1  10   1   1
   1   1  13   1   1   1   1   1  11   1   1   1   1  13   1   1   1   1
   1  13   1   1   1   1]
run_frames after: : [ 46  47  48  49  50  51  64  65  66  67  68  69  79  80  81  82  83  84
 198 199 200 201 202 203 213 214 215 216 217 228 229 230 231 232 242 243
 244 245 246 259 260 261 262 263 264 275 276 277 278 279 292 293 294 295
 296 297 310 311 312 313 314]
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [ ]:
print(len(avgs))
print(len(avgs))
print(avgs[0][0][8])
#print(avgs)

2
2
[-23.82484177 -51.85484177 -23.57484177 -13.85484177  22.27515823
  19.37515823  -9.29484177  -6.25484177  -5.61484177 -12.68484177
  -1.40484177   8.97515823  -1.45484177  11.94515823  19.00515823
  33.67515823  34.87515823  33.76515823  -7.68484177 -18.99484177
  -0.29484177  -9.29484177 -25.28484177 -45.98484177 -23.16484177
 -21.93484177 -17.94484177 -23.19484177 -26.36484177   0.74515823
 -11.22484177   3.25515823  -4.99484177  30.25515823  20.61515823
  12.83515823  12.55515823  -5.12484177 -11.82484177 -28.93484177
 -18.00484177  25.49515823  -1.47484177   5.84515823  21.38515823
   8.97515823 -13.12484177  17.77515823  24.78515823  52.68515823
  30.74515823  -4.82484177  29.27515823  30.51515823  59.69515823
  36.71515823  32.94515823]


In [ ]:
print(avgs[0][0])
#print(len(evs[3][1]['onset']))

[[-12.84082278 -47.04082278   8.75917722 ...  58.75917722  77.55917722
   76.75917722]
 [-23.12911392 -13.22911392 -45.92911392 ...  36.37088608  31.57088608
  -22.82911392]
 [-88.4085443  -58.4085443  -29.4085443  ... -15.9085443  -10.3085443
  -49.9085443 ]
 ...
 [ 17.62088608  47.62088608  50.42088608 ... -24.87911392  71.82088608
   32.42088608]
 [ 29.86708861 -73.63291139 146.76708861 ...  81.66708861  65.06708861
   94.66708861]
 [112.17689873  72.47689873  55.77689873 ... -28.12310127 -69.62310127
  -63.92310127]]


In [ ]:
print(evs[0][0])

{'onset': array([ 34.05 ,  47.431,  58.693, 143.991, 154.307, 166.648, 182.189,
       195.01 , 208.218, 221.746]), 'duration': array([3.675, 3.622, 3.625, 3.058, 3.155, 4.07 , 3.565, 3.716, 3.948,
       3.605]), 'amplitude': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}


In [ ]:
print(len(evs))

2


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0cfc2388-b22c-424d-b321-d09bf45f57a9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>